# SAP Developer Challenge – APIs
This notebook contains my solutions for the [SAP Developer Challenge – APIs](https://blogs.sap.com/2023/08/01/sap-developer-challenge-apis/).

In [ ]:
!pip3 install pyjwt
!pip3 install pyjwt[crypto]

In [ ]:
import requests
import json
import urllib.parse
import os
import pprint

COMMUNITY_ID = "ceedee666"
NW_BASE_URL = "https://developer-challenge.cfapps.eu10.hana.ondemand.com/odata/v4/northbreeze"
ACCOUNTS_SERVICE_URL = "https://accounts-service.cfapps.eu10.hana.ondemand.com"
ACCESS_TOKEN_URL = "https://christiandrumm.authentication.eu10.hana.ondemand.com" + "/oauth/token"

## Task 0 - Learn to share your task results
The task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-0-learn-to-share-your-task/m-p/276058#M2319).

In [ ]:
def get_hash_for_value(value, headers={"CommunityID" : COMMUNITY_ID}):
    url = f"https://developer-challenge.cfapps.eu10.hana.ondemand.com/v1/hash(value='{value}')"
    r = requests.get(url, headers=headers)
    return r.status_code, r.text

print(get_hash_for_value("this-is-the-year-of-the-api"))

## Task 1 - List the Northwind entity sets
The task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-1-list-the-northwind-entity/m-p/276626).

In [ ]:
def northwind_entity_sets():
    northwind_service_url = "https://services.odata.org/V4/Northwind/Northwind.svc/"
    r = requests.get(northwind_service_url)
    
    entity_sets = [ v["name"] for v in r.json()["value"] if v["kind"] == "EntitySet"]
    
    return sorted(entity_sets)

entity_sets = northwind_entity_sets()
print(f"The Northwind service provides {len(entity_sets)} entity sets.")

value = ",".join(entity_sets)
print("The entity sets provided by the Northwind service are:", value)
print("The resulting hash value is:", get_hash_for_value(value)[1])

## Task 2 - Calculate Northbreeze product stock 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-2-calculate-northbreeze/m-p/277325). For this task the we need to
- Calculate the sum of the `UnitsInStock` for all products
- Only products that are not discontinued (ie. `Discontinued: false`) should be counted.

I solved this task using three approaches:
1. Getting all data form the service and perform the filtering and aggregation in Python 🐍
2. Filtering the data using OData '$filter` and the aggregation in Python 🐍
3. Using OData data for filtering and aggregation as described [here](https://github.com/qmacro/odata-v4-and-cap/blob/main/slides.md#data-aggregation)


In [ ]:
PRODCUTS_URL = NW_BASE_URL + "/Products"

def product_stock_in_python():
    r = requests.get(PRODCUTS_URL)
    products = r.json()["value"]
    products_in_stock = {p["ProductID"] : p["UnitsInStock"] for p in products if p["Discontinued"] == False}
    return products_in_stock

def product_stock_with_odata_filter():
    r = requests.get(PRODCUTS_URL + "?$filter=Discontinued eq false&$select=ProductID,UnitsInStock")
    products = r.json()["value"]
    return {p["ProductID"] : p["UnitsInStock"] for p in products}

def product_stock_with_odata():
    r = requests.get(PRODCUTS_URL + "?$apply=filter(Discontinued eq false)/aggregate(UnitsInStock with sum as TotalStock)")
    return r.json()["value"][0]["TotalStock"]




print("Solution 1: Performating filtering and aggreation in Python:")
products_in_stock = product_stock_in_python()
print(f"The service returns {len(products_in_stock.keys())} that are not disontinued. The total stock of these products is {sum(products_in_stock.values())}")

print("\nSolution 2: Performating filtering in OData and aggreation in Python:")
products_in_stock = product_stock_with_odata_filter()
print(f"The service returns {len(products_in_stock.keys())} that are not disontinued. The total stock of these products is {sum(products_in_stock.values())}")

print("\nSolution 3: Performating filtering and aggregation in OData:")
total_stock = product_stock_with_odata()
print(f"The total stock of these products is {total_stock}")


print("\nThe resulting hash value is:", get_hash_for_value(total_stock)[1])

## Task 3 - Have a Northbreeze product selected for you 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-3-have-a-northbreeze-product/td-p/277972). For this task the we need to use the `selectProduct` action to let the service select a product based on our community id.

In [ ]:
SELECT_PRODUCT_URL = NW_BASE_URL + "/selectProduct"

def call_select_product():
    payload = {"communityid" : COMMUNITY_ID}
    r = requests.post(SELECT_PRODUCT_URL, json=payload)
    return r.json()["value"]

selected_product = call_select_product()
print(f"The Northbreeze service selected the product {selected_product} for my community ID")

encoded_product = urllib.parse.quote_plus(selected_product)
print(f"The URL-encoded product is: {encoded_product}")

print("\nThe resulting hash value is:", get_hash_for_value(encoded_product)[1])
    

## Task 4 - Discover the Date and Time API Package 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-4-discover-the-date-and-time/m-p/278745#M2897). For this task the we need to find all API endpoints that
- are accessible with the HTTP GET method
- return a response in JSON format.

In [ ]:
with open("DateAndTime.json") as f:
    api_spec = json.load(f)

paths = api_spec["paths"]

endpoints = sorted(
    filter(lambda p: paths[p]["get"]["produces"][0] == "application/json", 
           filter(lambda p: "get" in paths[p], paths)))

print(f"The API contains {len(endpoints)} endpoints")

string_for_hashing = ":".join(endpoints)
print(f"The string for hashing is: {string_for_hashing}")

print("\nThe resulting hash value is:", get_hash_for_value(string_for_hashing)[1])

## Task 5 - Call the country date format API endpoint 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-5-call-the-country-date-format/td-p/279160). For this task the we need to call the `getCountryDateFormat` endpoint of the date and time API. 
Additionally, we need to find a custom header that starts wit `x-` and is related to a SAP programming language. 

ABAP is not dead! ✌️

The service uses an API key for authentication. The following code expects that the API key is stored in an environment variable named `APIKEY`. 

In [ ]:
if "APIKEY" not in os.environ:
    print("Please set the environmet variable APIKEY to your API key from the API hub")

def call_get_country_date_format(country="DE"):
    URL = f"https://sandbox.api.sap.com/dateandtime/getCountryDateFormat?country={country}"
    r = requests.get(URL, headers={"APIKey":os.environ["APIKEY"]})
    hidden_headers = list(filter(lambda h: "abap" in h, r.headers))
    
    return r.text, hidden_headers[0]

format, header = call_get_country_date_format()

print("The endpoint returns the following format for the country DE: ", format)
print("The hidden header is: ", header)

print("\nThe resulting hash value is:", get_hash_for_value(format + "," + header)[1])

## Task 6 - Create a new Northbreeze category
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-6-create-a-new-northbreeze/m-p/279812#M3145). For this task a new category needs to be created in the northbreeze service.

In [ ]:
COMMUNITY_ID_NUM = 10851
CATEGORIES_URL = NW_BASE_URL + "/Categories"

def create_category(category_id = COMMUNITY_ID_NUM, category_name = COMMUNITY_ID, description = "August Developer Challenge"):
    category = {"CategoryID": category_id, "CategoryName": category_name, "Description": description }
    r = requests.post(CATEGORIES_URL, json=category)

def read_category(category_id = COMMUNITY_ID_NUM):
    r = requests.get(f"{CATEGORIES_URL}/{category_id}?$select=CategoryID,CategoryName")
    return r.status_code, r.text

create_category()
status_code, response = read_category()

print(f"The response returned by reading the category ig {COMMUNITY_ID_NUM} is: {response}")

encoded_response = urllib.parse.quote_plus(response)
print(f"\nThe URL-encoded response is: {encoded_response}")

print("\nThe resulting hash value is:", get_hash_for_value(encoded_response)[1])

## Task 7 - Create a new directory in an SAP BTP account 

The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-7-create-a-new-directory-in-an/m-p/280341). 

For this task I used the [SAP BTP Command Line Interface (btp CLI)](https://tools.hana.ondemand.com/#cloud).


In [ ]:
dir_id = "626141be-0712-4d0c-b9b2-521d8a27a651"

print("\nThe resulting hash value is:", get_hash_for_value(len(dir_id))[1])

## Task 8 - Create an instance of the SAP Cloud Management Service
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-8-create-an-instance-of-the/td-p/280681).

I used the BTP and CF CLIs to solve this task. The most difficult part was to login to my account (my email is related to two global accounts). Once I used `cf login --sso` it worked. 

In [ ]:
type = "managed_service_instance"

print("\nThe resulting hash value is:", get_hash_for_value(type)[1])

## Task 9 - Create a service key for API endpoints and auth info
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-task-9-create-a-service-key-for-api/m-p/281337). 

I used the CF CLI to create the service key. The name of my cis service instance is *my-cis*. The name of the created service key is *my-cis-sk*. The necessary commands to create and read the service key are 
```zsh
# create a service key
cf create-service-key my-cis my-cis-sk 

# read all service keys of service my-cis
cf service-keys my-cis  

# get details of service key my-cis-sk of service my-cis
cf service-key my-cis my-cis-sk 
```

In [ ]:
endpoints = { "accounts_service_url": "https://accounts-service.cfapps.eu10.hana.ondemand.com",
      "cloud_automation_url": "https://cp-formations.cfapps.eu10.hana.ondemand.com",
      "entitlements_service_url": "https://entitlements-service.cfapps.eu10.hana.ondemand.com",
      "events_service_url": "https://events-service.cfapps.eu10.hana.ondemand.com",
      "external_provider_registry_url": "https://external-provider-registry.cfapps.eu10.hana.ondemand.com",
      "metadata_service_url": "https://metadata-service.cfapps.eu10.hana.ondemand.com",
      "order_processing_url": "https://order-processing.cfapps.eu10.hana.ondemand.com",
      "provisioning_service_url": "https://provisioning-service.cfapps.eu10.hana.ondemand.com",
      "saas_registry_service_url": "https://saas-manager.cfapps.eu10.hana.ondemand.com" }

string_for_hashing = ",".join(sorted(endpoints.keys()))
print("\nThe resulting hash value is:", get_hash_for_value(string_for_hashing)[1])

## Task 10 - Request an OAuth access token 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-10-request-an-oauth-access/m-p/281933#M3586).

In [ ]:
def get_access_token(clientid, clientsecret, username, password):
    r = requests.post(ACCESS_TOKEN_URL, 
                      headers={"Content-Type": "application/x-www-form-urlencoded"},
                      auth=(clientid, clientsecret),
                      data={"grant_type": "password", "username": username, "password": password}
                      )
    return r.status_code, r.json()
    

if any([e not in os.environ for e in ["CLIENTID", "CLIENTSECRET", "BTPUSER", "BTPPASSWD"]]):
    print("Please set the environmet variables CLIENTID and CLIENTSECRET to the values provided by the service key. Set BTPUSER and BTPPASSWD to your user and password.")
else:
    status_code, result = get_access_token(os.environ["CLIENTID"], 
                                           os.environ["CLIENTSECRET"],
                                           os.environ["BTPUSER"], 
                                           os.environ["BTPPASSWD"])
    
    if status_code == 200:
        token_expires_in_s = result['expires_in']
        token_expires_in_h = round(token_expires_in_s / 3600)
        
        print(f"The access token expires in {token_expires_in_s} second. That are about {token_expires_in_h} hours.")
    
        print("The resulting hash value is:", get_hash_for_value(token_expires_in_h)[1])

## Task 11 - Examine the access token for scopes contained 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-11-examine-the-access-token/m-p/282366).

Failed to decode the token in Python using [PyJWT](https://pyjwt.readthedocs.io/en/stable/). Simply used sed the following command to decode the token: 

```
jq -r .access_token tokendata.json | jwt --output=json
```


In [ ]:
print("The resulting hash value is:", get_hash_for_value("RS256:JWT:23")[1])

In [ ]:
import jwt

def decode_jwt_token(token):
    header = jwt.get_unverified_header(token)
    
    keys_json = requests.get(header["jku"]).json()
    public_key = keys_json["keys"][0]["value"]
    payload = jwt.decode(token, public_key, header["alg"], options={"verify_aud": False})
    return header, payload

token = result["access_token"]
header, payload = decode_jwt_token(token)

pp = pprint.PrettyPrinter()

print("JWT Token Header:")
pp.pprint(header)
print("JWT Token Payload:")
pp.pprint(payload)

print("The resulting hash value is:", get_hash_for_value(f"{header['alg'] }:{header['typ']}:{len(payload['scope'])}")[1])

## Task 12 - Use the access token to call the API endpoint 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development-discussions/sap-developer-challenge-apis-task-12-use-the-access-token-to-call-the-api/td-p/283202).

In [ ]:
import time
from datetime import datetime

def refresh_access_token(clientid, clientsecret, refresh_token):
    r = requests.post(ACCESS_TOKEN_URL, 
                      headers={"Content-Type": "application/x-www-form-urlencoded"},
                      auth=(clientid, clientsecret),
                      data={"grant_type": "refresh_token", "refresh_token": refresh_token}
                      )
    return r.status_code, r.json()


def get_and_refresh_token():
    status_code, result = get_access_token(os.environ["CLIENTID"], 
                                           os.environ["CLIENTSECRET"],
                                           os.environ["BTPUSER"], 
                                           os.environ["BTPPASSWD"])
    token = result["access_token"]
    _, payload = decode_jwt_token(token)
    
    print("Expiration date of the access token:", datetime.fromtimestamp(payload["exp"]).strftime("%c"))
    
    print("waiting 10s....")
    time.sleep(10)
    
    print("Refreshing token....")
    status_code, result = refresh_access_token(os.environ["CLIENTID"], 
                                               os.environ["CLIENTSECRET"],
                                               result["refresh_token"])
    token = result["access_token"]
    _, payload = decode_jwt_token(token)
    print("Expiration date of the refreshed access token:", datetime.fromtimestamp(payload["exp"]).strftime("%c"))

    return token

def call_btp_apis(access_token):
    r = requests.get(ACCOUNTS_SERVICE_URL + "/accounts/v1/globalAccount?expand=true", 
                     headers={"authorization": f"bearer {access_token}"})

    dir_guid = r.json()["children"][0]["guid"]
   
    r = requests.get(ACCOUNTS_SERVICE_URL + f"/accounts/v1/directories/{dir_guid}", 
                     headers={"authorization": f"bearer {access_token}"})
    dir_data = r.json()
    return dir_data["displayName"], dir_data["directoryType"]

token = get_and_refresh_token()
dir_name, dir_type = call_btp_apis(token)

print("The resulting hash value is:", get_hash_for_value(f"{dir_name}:{dir_type}")[1])
    